<a href="https://colab.research.google.com/github/shreyasprabhakar/Major-Project/blob/main/ChurnXGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/Churn_Modelling 1.csv')

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,Delhi,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Chennai,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,Delhi,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,Delhi,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Chennai,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
df.duplicated().sum()

0

In [ ]:
df['Exited'].value_counts

<bound method IndexOpsMixin.value_counts of 0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64>

In [ ]:
df['Geography'].value_counts

<bound method IndexOpsMixin.value_counts of 0         Delhi
1       Chennai
2         Delhi
3         Delhi
4       Chennai
         ...   
9995      Delhi
9996      Delhi
9997      Delhi
9998     Mumbai
9999      Delhi
Name: Geography, Length: 10000, dtype: object>

In [ ]:
df['Gender'].value_counts

<bound method IndexOpsMixin.value_counts of 0       Female
1       Female
2       Female
3       Female
4       Female
         ...  
9995      Male
9996      Male
9997    Female
9998      Male
9999    Female
Name: Gender, Length: 10000, dtype: object>

In [ ]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace = True)

In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Delhi,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Chennai,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,Delhi,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,Delhi,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Chennai,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)

In [ ]:
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Delhi,Geography_Mumbai,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,1


In [ ]:
X = df.drop(columns=['Exited'])
y = df['Exited'].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Delhi,Geography_Mumbai,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,1


In [ ]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
X_train.shape

(8000, 11)

In [ ]:
X_test.shape

(2000, 11)

model building


In [ ]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
preds = xgb_cl.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, preds)
print('Model Accuracy for Test Dataset', acc)

Model Accuracy for Test Dataset 0.857


hyperparameter tuning

In [ ]:
param_grid = {
    "max_depth": [5],
    "learning_rate": [0, 0.01, 0.05, 0.1],
    "gamma": [1, 5, 10],
    "scale_pos_weight": [2, 5, 10, 20],
    "subsample": [1],
    "colsample_bytree": [1]
}
xgb_cl2 = xgb.XGBClassifier(objective="binary:logistic")
grid_cv = GridSearchCV(xgb_cl2, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")
_ = grid_cv.fit(X_train, y_train)
error_score='raise'
print("The Best Score:", grid_cv.best_score_)
print("The Best Params:", grid_cv.best_params_)


The Best Score: 0.8627400252510382
The Best Params: {'colsample_bytree': 1, 'gamma': 5, 'learning_rate': 0.1, 'max_depth': 5, 'scale_pos_weight': 2, 'subsample': 1}


In [ ]:
final_cl = xgb.XGBClassifier(
    **grid_cv.best_params_, objective="binary:logistic"
)
grid_final = final_cl.fit(X_train, y_train)
preds = grid_final.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Accuracy of the final model", acc)

Accuracy of the final model 0.8505
